##VGG19 model for FeelBeat with FER dataset

In [ ]:
!pip install keras-application

In [2]:
!pip install scikit-image
!pip install pydot


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from sklearn.metrics import *
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
import skimage
from skimage.transform import rescale, resize

import pydot

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
EPOCHS = 100
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 19

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197

In [6]:
vgg19_notop = VGG19(include_top =False,weights='imagenet',input_shape =(Resize_pixelsize, Resize_pixelsize,3), pooling='avg')
print(vgg19_notop.summary())
last_layer = vgg19_notop.get_layer('block4_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(4096, activation ='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)

for i in range(FROZEN_LAYER_NUM):
    vgg19_notop.layers[i].trainable = False

print(vgg19_notop.get_layer('block4_pool').trainable)

out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(vgg19_notop.input, out)


sgd = tf.keras.optimizers.legacy.SGD(learning_rate=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
callbacks_list = [rlrop]

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])



80134624/80134624 [==============================] - 0s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 197, 197, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 197, 197, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 197, 197, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 98, 98, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 98, 98, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 98, 98, 128)       147584    
                                                              

In [7]:
! rm -rf train; mkdir train
! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/train.zip' -d train
! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/ck_train.zip' -d train
! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/ck+_train.zip' -d train
! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/train_jaffe.zip' -d train



In [8]:
! rm -rf dev; mkdir dev
! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/test-public.zip' -d dev
!rm -rf test; mkdir test

! unzip -q '/content/drive/MyDrive/CSE499A/Datasets/test-private.zip' -d test

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(dataset, aug=False):
    if aug:
        datagen = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
            dataset,
            target_size=(197, 197),
            color_mode='rgb',
            shuffle = True,
            class_mode='categorical',
            batch_size=BS)

In [10]:
train_generator  = get_datagen('/content/train', True)
dev_generator  = get_datagen('/content/dev')
test_generator  = get_datagen('/content/test')

Found 30533 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
history = model.fit(
    x = train_generator,
    validation_data=test_generator,
    steps_per_epoch=28709// BS,
    validation_steps=3509 // BS,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    use_multiprocessing=True,
)

Epoch 1/100
224/224 [==============================] - ETA: 0s - loss: 243943.6250 - accuracy: 0.2413

224/224 [==============================] - 357s 2s/step - loss: 243943.6250 - accuracy: 0.2413 - val_loss: 1.8330 - val_accuracy: 0.2471 - lr: 0.0100
Epoch 2/100
224/224 [==============================] - ETA: 0s - loss: 1.8307 - accuracy: 0.2463

224/224 [==============================] - 361s 2s/step - loss: 1.8307 - accuracy: 0.2463 - val_loss: 1.8189 - val_accuracy: 0.2483 - lr: 0.0100
Epoch 3/100
224/224 [==============================] - ETA: 0s - loss: 1.8304 - accuracy: 0.2462

224/224 [==============================] - 373s 2s/step - loss: 1.8304 - accuracy: 0.2462 - val_loss: 1.8169 - val_accuracy: 0.2442 - lr: 0.0100
Epoch 4/100
 58/224 [======>.......................] - ETA: 4:19 - loss: 88.7787 - accuracy: 0.2485